In [1]:
import folium
from src.apikeys import MONGO
from src import process_tweets
from pymongo import MongoClient
from src.model import TweetPredictor
import pandas as pd

In [2]:
tweet_processor = process_tweets.TweetProcessor('../models/naive_bayes.pk')


In [3]:
client = MongoClient('mongodb://{}:{}@localhost:27017'.format(MONGO.USER, MONGO.PASSWORD))

In [4]:
database = client['twitter_database']

In [5]:
database.collection_names()

['test',
 'rainbows',
 'trends_2017_10_18_9',
 'trends_2017_10_16_17',
 'trends_2017_10_17_18',
 'emoticons',
 'broncos_raiders',
 'available_trends',
 'test_method',
 'trends_2017_10_17_19',
 'patriots_panthers',
 'trends_2017_10_17_17',
 'politics',
 'rep_dem_kasich',
 'roll_tide']

In [6]:
table = database['patriots_panthers']

In [7]:
df = tweet_processor.process_database(table, topics=['patriots'])

In [8]:
df.head(10)

,text,state,date,sentiment,time_bin
0,RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1,Virginia,2017-10-01 16:54:59+00:00,0.520838,0
1,@Patriots #PatsNation #NewEnglandPatriots #Do...,Florida,2017-10-01 16:54:59+00:00,0.520838,0
2,RT @Patriots: #TFW you hear #boomtower is back...,North Carolina,2017-10-01 16:55:00+00:00,0.466309,0
3,Theres always some extra hate in me whenever t...,North Carolina,2017-10-01 16:55:00+00:00,-0.503674,0
4,RT @MikeReiss: Videoboard shows Jerod Mayo ope...,Washington,2017-10-01 16:55:01+00:00,0.202868,0
5,RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1,Massachusetts,2017-10-01 16:55:02+00:00,0.520838,0
6,Watch Carolina Panthers vs New England Patriot...,Nevada,2017-10-01 16:55:03+00:00,0.822389,0
7,Carolina Panthers VS New England Patriots Live...,None,2017-10-01 16:55:04+00:00,0.822389,0
8,RT @Deacon_1971: POLL: Which POTUS loves Ameri...,Wisconsin,2017-10-01 16:55:04+00:00,0.620544,0
9,GAMETIME: Your New England Patriots look to im...,None,2017-10-01 16:55:03+00:00,0.640782,0


In [9]:
from src.visualization import visualize_sentiment
from src.visualization import visualize_count
from src.visualization import visualize_percent_diff
my_map = visualize_sentiment(df)
my_map

In [10]:
my_map.save('../website/maps/patriots_sentiment.html')

In [11]:
my_map, _ = visualize_percent_diff(df)
my_map

In [13]:
my_map.save('../website/maps/patriots_count.html')

In [13]:
import numpy as np
a = df.groupby('state')['sentiment'].agg(['mean','std','count'])
a['mean_dev'] = a.apply(lambda row: row['std']/np.sqrt(row['count']),axis=1)
a[['mean','mean_dev']].head(50)

,mean,mean_dev
state,,
Alabama,0.208878,0.023047
Alaska,0.276945,0.056105
Arizona,0.272656,0.020517
Arkansas,0.210744,0.039092
California,0.235440,0.009546
Colorado,0.182623,0.025362
Connecticut,0.184677,0.014257
Delaware,0.270958,0.013619
District of Columbia,0.184265,0.044520


In [14]:
for i in df[df.state=='Massachusetts'][['text','sentiment']].values:
    print(i[0],i[1])

RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1 0.5208384594279665
Intrigante saber como responderá la defensiva de #patriots hoy vs #panters. estan en el sótano de la NFL 0.0
Intrigante saber como responderá la defensiva de #patriots hoy vs #panters. estan en el sótano de la NFL 0.0
#Patriots time!!!!! #GoPats #PatriotsNation 0.04829677345400141
Moss names Brady the best QB of all time - ESPN Video #Patriots  https://t.co/DY3oKl4MLp https://t.co/7KBzd7EGxU 0.8841541285497012
RT @NEPatsNation12: GAMETIME: Your New England Patriots look to improve their record to 3-1 against the Carolina Panthers! https://t.co/dJV… 0.746596434819969
MATT .@mattbare23 Sorry  your request came late to me : I already put out the #Patriots as my best play if they win… https://t.co/B1jHcbVGzN -0.1778591989766548
RT @Patriots: It's game day in Foxboro! https://t.co/2tQ13eG8Xk 0.5369816254710345
RT @MarkDanielsPJ: Looks like the Patriots will stand together for the anthem 0.27780573625682337
Going to be a good

RT @RyanHannable: Jerod Mayo opens the door to the Patriots locker room and Brady gives him a big hug before coming out to tunnel. 0.1577421976167046
RT @Patriots: #SG3 gets the #Patriots on the board. https://t.co/ZJDnICNP5W 0.5208384594279665
RT @TomLeyden: #Patriots all stand, hand on heart - left hand on teammates shoulders. https://t.co/tqlAeHN6FS 0.11303627707393371
Early indication of how Patriots plan to utilize LB Dont'a Hightower today: He's on in base package but comes off in passing situations. -0.3445046597798075
The Patriots have too much talent on defense to be THIS BAD!!!! -0.657253884284352
RT @jeffphowe: The Patriots are standing in line for the anthem with their right hands over their hearts and their left hands on their team… -0.32189273892127235
#Patriots defense is terrible 0.0
Two plays in and I'd be fooled that Cam Newton isn't playing well. Two easy completions #Patriots 0.09969592620488421
Patriots D is last in the league. Wonder if Patricia knows that?..Doesn

SCORE UPDATE: Patriots 13, Panthers 17. Follow the game live on https://t.co/fuqrt6GLL6 #Patriots 0.6647920522409132
RT @leahysean: Live updates: #Patriots defense is breaking down after back-to-back TD drives for Carolina. Panthers 17, Pats 13 https://t.c… 0.4761795171422578
RT @leahysean: Live updates: #Patriots defense is breaking down after back-to-back TD drives for Carolina. Panthers 17, Pats 13 https://t.c… 0.4761795171422578
RT @jeffphowe: Rowe’s missed tackle earlier in the drive cost the Patriots 7 points (plus a full possession for Brady). -0.5400810817001377
This Patriots Defence is pathetic. -0.1156226390975994
Are the Patriots gonna cover or no? -0.09991309096352097
#Panthers are making the #Patriots defense look like a hot mess, worse than it already was. #CARvsNE https://t.co/MGOybl8RrZ -0.037505119988156865
Sucks #Patriots it's True. https://t.co/V2biloAM1d -0.5507740573814834
Patriots defense trying very hard to maintain their status as the worst in the league!! https

#Patriots defense is absolutely EMBARRASSING. #Panthers score again Newton to Funchess. at least they shanked the extra point. 23-16 CAR -0.3573952057905
RT @LifeOfThad: My #Patriots defense is so damn Stressful smh !!!! Like how u letting Cam Newton do this shit smh -0.4653722692067408
@scammyjenkins Gilmore has taken a massive step back. How exactly is he worse on the patriots than the bills? 0.053424461125638256
Are you a bum wide receiver? Well you're going to love it when you play the Patriots hapless secondary #Patriots #Panthers #NFL 0.7582214312739473
RT @PatriotsSB49: That 2011 Patriots defense sucked!

2017 Patriots: “Hold my beer.” -0.634529068006457
RT @Patriots: Panthers lead 23-16 following a 16-yard TD catch by Funchess. PAT is wide left. #CARvsNE 0.22249013310853694
@Patriots How does matt patricia still have a job? -0.023152781718942195
RT @BosSportsExtra: RT and follow @OYOSports to win this #Patriots Endzone. Winner will be chosen after today's game. https://t.co/gRP

Rob Gronkowski now has a special place in Patriots’ history https://t.co/UTa2Rts9NG https://t.co/1dr7eRIcic 0.5308226420524091
As frustrating and painful as it can be watching the patriots this is why you don't turn them off -0.6501849409539617
@PatriotsSB49 love having him as our cushion without Edelman, plays with almost as much heart and toughness 0.6503492424912629
So close… #CARvsNE #GoPats @Patriots #Patriots 0.20286776979717214
Tom Brady has passed for more than 300 yards in a game for the 79th time in his career. Third most all time. #Patriots #NFL 0.10779908227293222
#FreeDion with another red-zone carry gets 5 to the Carolina 1. #CARvsNE #Patriots -0.12276058315270111
It must be just as nerve wracking to be up on the Patriots as down to them if it's the 4th quarter. -0.03937016523922132
Patriots going two tight ends on this next play. -0.29197012460915794
Fantasy football sit/start, Week 4: What to do with Christian McCaffrey: https://t.co/fegRgqlo3A #Patriots 0.1788408686903

In [ ]:
df.get_values()

In [14]:
table = database['roll_tide']
df = tweet_processor.process_database(table)

In [15]:
from src.visualization import visualize_sentiment
from src.visualization import visualize_count
my_map, sentiment_score = visualize_percent_diff(df)
print(sentiment_score)
my_map

0.31735543162954444
